In [122]:
import numpy as np
import pandas as pd
import sympy as sp
from sympy import pprint

In [75]:
df = pd.read_excel('Teste.xlsx',header=None)
df
# número de nós
# ID / x / y / restrição em x / restrição em y / restrição momento
# número de materiais
# ID / módulo de elasticidade
# número de barras
# ID / nó inicial / nó final / secão x / seção y / E
# número de cargas em nós
# nó / valor da carga x / valor da carga y
# número de cargas em barras
# barra / direção / inicio da carga / fim da carga / valor da carga no inicio / valor da carga no fim

,0,1,2,3,4,5
0,3,NaN,NaN,NaN,NaN,NaN
1,1,0.0,0.0,1.0,1.0,1.0
2,2,0.0,4.0,0.0,0.0,0.0
3,3,6.0,4.0,1.0,1.0,1.0
4,1,NaN,NaN,NaN,NaN,NaN
5,1,1.0,NaN,NaN,NaN,NaN
6,2,NaN,NaN,NaN,NaN,NaN
7,1,1.0,2.0,1.0,1.0,1.0
8,2,2.0,3.0,1.0,1.0,1.0
9,2,NaN,NaN,NaN,NaN,NaN


In [76]:


class No:
    def __init__(self, x, y, Rx, Ry, Rm):

        self.x = x          #posição x do nó
        self.y = y          #posição y do nó
        self.Rx = Rx        #apoio na direção x (0 sem apoio, 1 com apoio)
        self.Ry = Ry        #apoio na direção y (0 sem apoio, 1 com apoio)
        self.Rm = Rm        #apoio em relação ao momento (0 sem apoio, 1 com apoio)
        self.carga = [0,0,0]

    def __str__(self):
        return f'x: {self.x} y: {self.y} Rx: {self.Rx} Ry: {self.Ry} Rm: {self.Rm} carga: {self.carga}'

    def adicionarcarga(self,carga):
      self.carga[0] = self.carga[0] + carga.forcax
      self.carga[1] = self.carga[1] + carga.forcay
      self.carga[2] = self.carga[2] + carga.forcamomento







class Material:
    def __init__(self, E):
      self.E = E            #módulo de elasticidade








class Barra:
    def __init__(self, no1, no2, secaox, secaoy, E=1):

      self.no1 = no1
      self.no2 = no2
      self.secaox = secaox
      self.secaoy = secaoy
      self.L = calculacomprimento(no1,no2)
      self.I = calculainercia(secaox,secaoy)
      self.Area = calculaarea(secaox,secaoy)
      self.E = E

      self.anguloeixox = calculaangulo(self)

    def __str__(self):
        return f'no1: {self.no1} \n\n no2: {self.no2}\n\n secaox: {self.secaox} secaoy: {self.secaoy} Comprimento: {self.L} I: {self.I} Area: {self.Area}'



        


class Carga:
    def __init__(self, no, forcax, forcay, forcamomento):
      self.forcax = forcax
      self.forcay = forcay
      self.forca = np.sqrt(pow(forcax,2) + pow(forcay,2))
      self.forcamomento = forcamomento
      self.no = no
      self.posicaox = no.x  #é passada a posição da carga através da posição do nó onde a carga foi aplicada
      self.posicaoy = no.y   # é necessário atualizar para possibilitar a utilização de outros tipos de carga,
                             # como carga distribuída, ou uma carag não aplicada em um nó







def calculaarea(secaox,secaoy):
  return secaox*secaoy



def calculainercia(secaox,secaoy):
  return secaox*(secaoy**3)/12



def calculacomprimento(no1,no2):

  return np.sqrt(pow(abs(no1.x - no2.x),2) + pow(abs(no1.y - no2.y),2))

def calculaangulo(barra):
  return np.arctan((barra.no2.y - barra.no1.y)/(barra.no2.x - barra.no1.x)) #calcula o angulo da barra em relação ao eixo x

In [77]:
class estrutura:
	def __init__(self, df):
		self.nos = {}
		self.barras = {}
		self.materiais = {}

		# leitura dos nós
		pos = 0
		num_nos = df.iloc[pos, 0]
		pos = pos + 1

		for i in range(1, num_nos+1):
			self.nos[df.iloc[pos, 0]] = No(df.iloc[pos, 1], df.iloc[pos, 2], df.iloc[pos, 3], df.iloc[pos, 4], df.iloc[pos, 5])
			pos += 1

		# leitura dos materiais
		num_materiais = df.iloc[pos, 0]
		pos = pos + 1
		self.materiais[df.iloc[pos, 0]] = Material(df.iloc[pos, 1])
		pos += 1
		


		# leitura das barras
		num_barras = df.iloc[pos, 0]
		pos = pos + 1

		for i in range(1, num_barras+1):
			self.barras[df.iloc[pos, 0]] = Barra(self.nos[df.iloc[pos, 1]], self.nos[df.iloc[pos, 2]], df.iloc[pos, 3], df.iloc[pos, 4])
			pos += 1

		# leitura das cargas nos nós
		num_cargas = df.iloc[pos, 0]
		pos = pos + 1

		for i in range(1, num_cargas+1):
			x = Carga(self.nos[df.iloc[pos, 0]], df.iloc[pos, 1], df.iloc[pos, 2], df.iloc[pos, 3])	
			
			x.no.adicionarcarga(x)
			pos += 1

	def calculaliberdade(Nos):
		grauliberdade = 0
		for no in Nos:
			if no.Rx == 1:
				grauliberdade += 1
			if no.Ry == 1:
				grauliberdade += 1
			if no.Rm == 1:
				grauliberdade += 1

		return grauliberdade - 3

	def __str__(self):
		st = 'Estrutura'
		st += '\nNos:'
		for no in self.nos:
			st += '\n' + str(no) + ': ' + str(self.nos[no])
		st += '\nBarras:'
		for barra in self.barras:
			st += '\n' + str(barra) + ': ' + str(self.barras[barra])
			st += '\n'
		return st
		

			
		




In [78]:
struct = estrutura(df)

for i in struct.nos:
    print(struct.nos[i].carga)
    
print(calculacomprimento(struct.nos[1],struct.nos[2]))
print(struct)

[0, 0, 0]
[10.0, -6.0, 0.0]
[0, 0, 0]
4.0
Estrutura
Nos:
1: x: 0.0 y: 0.0 Rx: 1.0 Ry: 1.0 Rm: 1.0 carga: [0, 0, 0]
2: x: 0.0 y: 4.0 Rx: 0.0 Ry: 0.0 Rm: 0.0 carga: [10.0, -6.0, 0.0]
3: x: 6.0 y: 4.0 Rx: 1.0 Ry: 1.0 Rm: 1.0 carga: [0, 0, 0]
Barras:
1: no1: x: 0.0 y: 0.0 Rx: 1.0 Ry: 1.0 Rm: 1.0 carga: [0, 0, 0] 

 no2: x: 0.0 y: 4.0 Rx: 0.0 Ry: 0.0 Rm: 0.0 carga: [10.0, -6.0, 0.0]

 secaox: 1.0 secaoy: 1.0 Comprimento: 4.0 I: 0.08333333333333333 Area: 1.0

2: no1: x: 0.0 y: 4.0 Rx: 0.0 Ry: 0.0 Rm: 0.0 carga: [10.0, -6.0, 0.0] 

 no2: x: 6.0 y: 4.0 Rx: 1.0 Ry: 1.0 Rm: 1.0 carga: [0, 0, 0]

 secaox: 1.0 secaoy: 1.0 Comprimento: 6.0 I: 0.08333333333333333 Area: 1.0



/var/folders/hk/51wclgdn5r98p8hh6ss6qzx80000gn/T/ipykernel_31681/1308753180.py:90: RuntimeWarning: divide by zero encountered in scalar divide
  return np.arctan((barra.no2.y - barra.no1.y)/(barra.no2.x - barra.no1.x)) #calcula o angulo da barra em relação ao eixo x


In [123]:
from sympy import symbols
from sympy.abc import E,A,L,I,theta
sp.init_printing(use_unicode=True)

#matriz de rigidez de uma barra
K = sp.Matrix([[E*A/L       ,0            ,0           ,-E*A/L     ,0            ,0          ],
               [0           ,12*E*I/L**3  ,6*E*I/L**2  ,0          ,-12*E*I/L**3 ,6*E*I/L**2 ],
               [0           ,6*E*I/L**2   ,4*E*I/L     ,0          ,-6*E*I/L**2  ,2*E*I/L    ],
               [-E*A/L      ,0            ,0           ,E*A/L      ,0            ,0          ],
               [0           ,-12*E*I/L**3 ,-6*E*I/L**2 ,0          ,12*E*I/L**3  ,-6*E*I/L**2],
               [0           ,6*E*I/L**2   ,2*E*I/L     ,0          ,-6*E*I/L**2  ,4*E*I/L    ]])

In [135]:
print(struct.barras[1].anguloeixox)

1.5707963267948966


In [151]:


def calculamatrizlocal(barra):
    restricoes1 = [barra.no1.Rx,barra.no1.Ry,barra.no1.Rm]
    restricoes2 = [barra.no2.Rx,barra.no2.Ry,barra.no2.Rm]

    K_local = K.subs([(E,barra.E),(A,barra.Area),(L,barra.L),(I,barra.I),(theta,barra.anguloeixox)])
    #rotaciona a matriz de rigidez local
    T = sp.Matrix([[sp.cos(theta) ,sp.sin(theta) ,0             ,0             ,0             ,0],
                   [-sp.sin(theta),sp.cos(theta) ,0             ,0             ,0             ,0],
                   [0             ,0             ,1             ,0             ,0             ,0],
                   [0             ,0             ,0             ,sp.cos(theta) ,sp.sin(theta) ,0],
                   [0             ,0             ,0             ,-sp.sin(theta),sp.cos(theta) ,0],
                  [0             ,0             ,0             ,0             ,0             ,1]])
    rotacao = T.subs([(theta,barra.anguloeixox)])
    K_local = rotacao.T*K_local*rotacao


    for i in range(0,3):
        if restricoes1[i] == 1:
            for j in range(0,6):
                K_local[i,j] = 0
                K_local[j,i] = 0
        if restricoes2[i] == 1:
            for j in range(0,6):
                K_local[i+3,j] = 0
                K_local[j,i+3] = 0

    for i in range(0,6):
        for j in range(0,6):
            if K_local[i,j] < 10**-14:
                K_local[i,j] = 0

    #aplica as restrições
    return K_local
    
    

In [155]:
K_local = {}
for i in struct.barras:
    K_local[i] = calculamatrizlocal(struct.barras[i])

dimensao = 3 + 3 * len(struct.barras)

K_global = sp.zeros(dimensao,dimensao)

for i, barra in enumerate(struct.barras):
    for j in range(0,6):
        for k in range(0,6):
            K_global[j + 3  *i, k + 3 * i] = K_global[j + 3 * i, k + 3 * i] + K_local[barra][j,k]

pprint(K_global)

⎡0  0  0          0                  0                   0           0  0  0⎤
⎢                                                                           ⎥
⎢0  0  0          0                  0                   0           0  0  0⎥
⎢                                                                           ⎥
⎢0  0  0          0                  0                   0           0  0  0⎥
⎢                                                                           ⎥
⎢0  0  0  0.182291666666667          0                0.03125        0  0  0⎥
⎢                                                                           ⎥
⎢0  0  0          0           0.25462962962963   0.0138888888888889  0  0  0⎥
⎢                                                                           ⎥
⎢0  0  0       0.03125       0.0138888888888889  0.138888888888889   0  0  0⎥
⎢                                                                           ⎥
⎢0  0  0          0                  0                   0      